## 0. Imports and Constants
- Select user before proceeding

In [3]:
############## AUTORELOAD MAGIC ###################
%load_ext autoreload
%autoreload 2
###################################################

############## FUNDAMENTAL MODULES ################
import json
from PIL import Image
import os
import sys
import copy
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
 ##################################################

############## TASK-SPECIFIC MODULES ##############
sys.path.append(os.path.join(os.getcwd(),"src"))
from vanilla_dataset import VanillaDataset
from vanilla_nn import VanillaNN
from trainer import Trainer
###################################################


####################### CONSTANTS ########################
SPLITS = ["train", "dev", "test"]
TRAIN, DEV, TEST, TXT, IMG = "train", "dev", "test", "txt", "img"
FE_METHODS = ["txt_embeddings", "img_embeddings", "concat", "sum", "mean", "hadamard"]
#FE_METHODS += ["concat_cos", "sum_cos", "mean_cos", "hadamard_cos"]
##########################################################

############## DATA SCIENCE & ML MODULES #################
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel
import torch
import pandas as pd
from scipy import stats
from sklearn.svm import SVC
from sklearn.metrics import classification_report
##########################################################

####################### SELECT ###########################
users = ["patriziopalmisano", "onurdenizguler", "jockl"]
user = users[1] # SELECT USER
version = "v2" # SELECT DATASET VERSION
dataset_version = version
##########################################################

if user in users[:2]:
    data_dir = f"/Users/{user}/Library/CloudStorage/GoogleDrive-check.worthiness@gmail.com/My Drive/data/CT23_1A_checkworthy_multimodal_english_{version}"
    cw_dir = f"/Users/{user}/Library/CloudStorage/GoogleDrive-check.worthiness@gmail.com/My Drive"

else:
    data_dir = f"/home/jockl/Insync/check.worthiness@gmail.com/Google Drive/data/CT23_1A_checkworthy_multimodal_english_{dataset_version}"
    cw_dir = "/home/jockl/Insync/check.worthiness@gmail.com/Google Drive"

features_dir = f"{data_dir}/features"
labels_dir = f"{data_dir}/labels"
models_dir = f"{cw_dir}/models/vanillannWithVal"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load the Datasets

In [2]:
split_to_labels = {split: 
                   np.load(f"{labels_dir}/{split}_labels_{dataset_version}.pickle", allow_pickle=True) 
                   for split in SPLITS}

method_to_split_to_data = {fe_method: {
                                    split: 
                                    np.load(f"{features_dir}/{fe_method}/{fe_method}_{split}_{dataset_version}.pickle", allow_pickle=True) 
                                    for split in SPLITS
                                    } 
                            for fe_method in FE_METHODS}

method_to_split_to_dataset = {fe_method: {
                                        split:
                                        VanillaDataset(method_to_split_to_data[fe_method][split], split_to_labels[split]) 
                                        for split in SPLITS
                                        } 
                                for fe_method in FE_METHODS}

# Perform hyperparameter search on a single model architecture with a selected method

In [29]:
# Select method 
dataset_dict = method_to_split_to_dataset["txt_embeddings"]
input_dim = len(dataset_dict["train"][0][0])
output_dim = 1

# Select model architecture and model class
layer_sizes = [input_dim, 64, 32, output_dim]
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer_1 = Trainer(model, layer_sizes, dataset_dict)